## Database tree

    database
    ├───subsys_00
    │   ├───temperature
    │   │   ├───temp_sensor_00
    │   │   └───temp_sensor_01
    │   └───luminosity
    └───subsys_01
        ├───temperature
        │   ├───temp_sensor_00
        │   └───temp_sensor_01
        └───luminosity

## Initial setup

In [ ]:
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import requests
import time

In [ ]:
def get_content(url, proxies=None, auth=None):
    req = requests.get(url, proxies=proxies, auth=auth)
    if req.status_code == 200:
        return req.content

    raise Exception(f"Request not successful, code {req}")

In [ ]:
proxies = {
    "http": "http://<url>:<port>",
    "https": "http://<url>:<port>"
}

proxy_auth = requests.auth.HTTPProxyAuth("<user>", "<password>")

firebase_host = "https://<database-name>.firebaseio.com"
url = firebase_host + "<database-name>.json"

## Read current data

In [ ]:
content = get_content(url, proxies, proxy_auth)
data = json.loads(content)

data_len = len(data)
indexes = np.array([int(x[-2:]) for x in data.keys()], dtype=np.int32)
data_max_len = indexes[-1]

luminosity = np.full(data_len, np.nan, dtype=np.float64)
temp_sensor_00 = np.full(data_len, np.nan, dtype=np.float64)
temp_sensor_01 = np.full(data_len, np.nan, dtype=np.float64)

for i, j in zip(range(data_len), indexes):
    try:
        luminosity[i] = data[f"subsys_{j:02}"]["luminosity"]
        temp_sensor_00[i] = data[f"subsys_{j:02}"]["temperature"]["temp_sensor_00"]
        temp_sensor_01[i] = data[f"subsys_{j:02}"]["temperature"]["temp_sensor_01"]
    except KeyError:
        pass

## Visualization

In [ ]:
plt.plot(indexes[~np.isnan(luminosity)], luminosity[~np.isnan(luminosity)], linestyle='', marker='o', label="data")
plt.plot(indexes, [np.mean(luminosity[~np.isnan(luminosity)])] * data_len, linestyle='-', marker='', label="mean")
plt.grid()
plt.xlabel("index")
plt.ylabel("luminosity (V)")
plt.show()

plt.plot(indexes[~np.isnan(temp_sensor_00)], temp_sensor_00[~np.isnan(temp_sensor_00)], linestyle='', marker='o',
         label="data")
plt.plot(indexes, [np.mean(temp_sensor_00[~np.isnan(temp_sensor_00)])] * data_len, linestyle='-', marker='',
         label="mean")
plt.grid()
plt.xlabel("index")
plt.ylabel("temp_sensor_00 (oC)")
plt.show()

plt.plot(indexes[~np.isnan(temp_sensor_01)], temp_sensor_01[~np.isnan(temp_sensor_01)], linestyle='', marker='o',
         label="data")
plt.plot(indexes, [np.mean(temp_sensor_01[~np.isnan(temp_sensor_01)])] * data_len, linestyle='-', marker='',
         label="mean")
plt.grid()
plt.xlabel("index")
plt.ylabel("temp_sensor_01 (oC)")
plt.show()

## Dashboard

In [ ]:
def get_means():
    content = get_content(url, proxies, proxy_auth)
    data = json.loads(content)
    
    data_len = len(data)
    indexes = np.array([int(x[-2:]) for x in data.keys()], dtype=np.int32)
    data_max_len = indexes[-1]
    
    luminosity = np.full(data_len, np.nan, dtype=np.float64)
    temp_sensor_00 = np.full(data_len, np.nan, dtype=np.float64)
    temp_sensor_01 = np.full(data_len, np.nan, dtype=np.float64)
    
    for i, j in zip(range(data_len), indexes):
        try:
            luminosity[i] = data[f"subsys_{j:02}"]["luminosity"]
            temp_sensor_00[i] = data[f"subsys_{j:02}"]["temperature"]["temp_sensor_00"]
            temp_sensor_01[i] = data[f"subsys_{j:02}"]["temperature"]["temp_sensor_01"]
        except KeyError:
            pass
        
    luminosity_mean = np.mean(luminosity[~np.isnan(luminosity)])
    temp_sensor_00_mean = np.mean(temp_sensor_00[~np.isnan(temp_sensor_00)])
    temp_sensor_01_mean = np.mean(temp_sensor_01[~np.isnan(temp_sensor_01)])
        
    return luminosity_mean, temp_sensor_00_mean, temp_sensor_01_mean

In [ ]:
%matplotlib notebook

fig, axs = plt.subplots(nrows=3, sharex=True, figsize=(16, 8), gridspec_kw={"hspace": 0.4})
fig.supxlabel("Time")
ax_luminosity, ax_temp_sensor_00,ax_temp_sensor_01 = axs

ax_luminosity.grid(True)
ax_luminosity.set_ylabel("Luminosity Mean")

ax_temp_sensor_00.grid(True)
ax_temp_sensor_00.set_ylabel("Temperature S0 Mean (oC)")

ax_temp_sensor_01.grid(True)
ax_temp_sensor_01.set_ylabel("Temperature S1 Mean (oC)")

plt.ion()
fig.show()
fig.canvas.draw()

while True:
    luminosity_mean, temp_sensor_00_mean, temp_sensor_01_mean = get_means()
    current_time = datetime.datetime.now()
    
    ax_luminosity.plot(current_time, luminosity_mean, linestyle='', marker='o', markersize=4, color='r')
    ax_temp_sensor_00.plot(current_time, temp_sensor_00_mean, linestyle='', marker='o', markersize=4, color='r')
    ax_temp_sensor_01.plot(current_time, temp_sensor_01_mean, linestyle='', marker='o', markersize=4, color='r')
    
    fig.canvas.draw()
    time.sleep(15)